In [1]:
# 🔄 Auto-reload modules when their source code changes
%load_ext autoreload
%autoreload 2

# 📦 Set up src/ and utils/ paths (project root is detected via pyproject.toml)
%run ../bootstrap.py

## Objective:
Reconstruct Oura-style nightly sleep summary for a single date
using v2 API endpoints.

In [2]:
import requests
from datetime import datetime, timedelta, timezone
import json
from pathlib import Path
from dotenv import load_dotenv
import os
import time
from pprint import pprint
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import display, clear_output

from sleeping_beauty.config.config import Config

In [3]:
load_dotenv()

True

In [4]:
config = Config()


PROJECT_ROOT = Path(config.PROJECT_ROOT)
config.config_path = str(PROJECT_ROOT / "configs" / "config.yaml")
# "configs/FE/features_config.yaml"
config.load_from_yaml(config.config_path)




[Config] Loaded YAML config: /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
[Config] Overriding 'divider': False → True
[Config] Setting 'divider': False → True


In [5]:
config.print_config_info()

📂 Configuration
--------------------------------------------------
Configuration file:       /Users/kenneth/Public/projects/python/observatory/sleeping-beauty/configs/config.yaml
--------------------------------------------------
🔐 Auth / Oura
--------------------------------------------------
Token path:               /Users/kenneth/.sleeping_beauty/oura_token.json
Client ID set:            True
Client Secret set:        True
Redirect URI:             http://localhost:8400/callback
Scopes:                   ['daily', 'heartrate', 'personal', 'session']


In [6]:
from datetime import date, timedelta
import asyncio

from sleeping_beauty.clients.oura_api_client import OuraApiClient
from sleeping_beauty.oura.auth.oura_auth import OuraAuth  
from sleeping_beauty.oura.auth.storage.file_storage import FileTokenStorage

In [13]:
from sleeping_beauty.oura.auth.domain.auth_preflight_result import AuthPreflightReport


storage = FileTokenStorage(
    path=Path(
        config.oura_token_path
    ).expanduser()
)

# Instantiate your existing auth service
oura_auth = OuraAuth.from_config()
preflight: AuthPreflightReport = oura_auth.preflight_check()
print("\n".join(preflight.messages))

# Token provider callable (what the client expects)
token_provider = oura_auth.get_access_token   ## notice this is passing the get_access_token as the callable function 

# Create the API client
client = OuraApiClient(token_provider=token_provider)

🔐 Oura Auth Preflight Check
----------------------------------------
✅ client_id present
✅ client_secret present
✅ redirect_uri: http://localhost:8400/callback
✅ requested scopes: ['extapi:daily', 'extapi:heartrate', 'extapi:personal', 'extapi:session']
✅ token found in storage
✅ token valid for ~42008 minutes
✅ token covers requested scopes
----------------------------------------
✔️  Preflight check complete


In [8]:
TARGET_DAY = date(2026, 1, 11)
TIMEZONE = "Europe/Luxembourg"

In [9]:
from datetime import timedelta, time, date

# ================================================================
# Helpers
# ================================================================


def seconds_to_hm(seconds: int | None) -> str:
    if not seconds or seconds <= 0:
        return "0h 00m"
    h = seconds // 3600
    m = (seconds % 3600) // 60
    return f"{h}h {m:02d}m"


def is_night_sleep(doc) -> bool:
    """
    Night sleep heuristic:
    Starts after 18:00 OR before noon (cross-midnight).
    """
    start = doc.bedtime_start.timetz()
    return start >= time(18, 0) or start <= time(12, 0)


def select_core_sleep(sleep_docs, target_day: date):
    """
    Canonical core sleep selection:
    1. Must end on target_day
    2. Prefer type == long_sleep
    3. Otherwise longest night sleep
    4. Otherwise longest episode
    """

    ending_today = [d for d in sleep_docs if d.day == target_day]
    assert ending_today, f"No sleep episodes ending on {target_day}"

    long_sleeps = [d for d in ending_today if d.type == "long_sleep"]
    if long_sleeps:
        return max(long_sleeps, key=lambda d: d.total_sleep_duration)

    night_sleeps = [d for d in ending_today if is_night_sleep(d)]
    if night_sleeps:
        return max(night_sleeps, key=lambda d: d.total_sleep_duration)

    return max(ending_today, key=lambda d: d.total_sleep_duration)


def compute_supplemental_sleep(sleep_docs, core_sleep):
    """
    Any sleep not selected as core counts as supplemental (naps).
    """
    supplemental = [
        d
        for d in sleep_docs
        if d.id != core_sleep.id and (d.total_sleep_duration or 0) > 0
    ]
    total_seconds = sum(d.total_sleep_duration for d in supplemental)
    return supplemental, total_seconds


# ================================================================
# 1. Fetch all sleep episodes around TARGET_DAY
#    (Oura sleep is keyed by END date)
# ================================================================

sleep_docs = []

client = OuraApiClient(token_provider=token_provider)

async for doc in client.iter_sleep(
    start_date=TARGET_DAY - timedelta(days=1),
    end_date=TARGET_DAY + timedelta(days=1),
):
    sleep_docs.append(doc)

assert sleep_docs, "No sleep documents found in expanded window"


# ================================================================
# 2. Select CORE sleep
# ================================================================

core_sleep = select_core_sleep(sleep_docs, TARGET_DAY)


# ================================================================
# 3. Fetch Daily Sleep Score (authoritative)
# ================================================================

daily_sleep_docs = []

client = OuraApiClient(token_provider=token_provider)

async for doc in client.iter_daily_sleep_scores(
    start_date=TARGET_DAY,
    end_date=TARGET_DAY,
):
    daily_sleep_docs.append(doc)

assert len(daily_sleep_docs) == 1, "Expected exactly one DailySleepScore"

daily_sleep = daily_sleep_docs[0]


# ================================================================
# 4. Fetch Daily Readiness Score
# ================================================================

readiness_docs = []

client = OuraApiClient(token_provider=token_provider)

async for doc in client.iter_daily_readiness_scores(
    start_date=TARGET_DAY,
    end_date=TARGET_DAY,
):
    readiness_docs.append(doc)

assert len(readiness_docs) == 1, "Expected exactly one DailyReadinessScore"

readiness = readiness_docs[0]


# ================================================================
# 5. Supplemental sleep + totals
# ================================================================

supplemental_sleeps, supplemental_seconds = compute_supplemental_sleep(
    sleep_docs, core_sleep
)

total_24h_seconds = core_sleep.total_sleep_duration + supplemental_seconds


# ================================================================
# 6. Final Summary Output
# ================================================================

night_label = (
    f"{core_sleep.bedtime_start.strftime('%b %d')} → "
    f"{core_sleep.bedtime_end.strftime('%b %d')}"
)

summary = f"""
🛏️ Sleep Summary — {core_sleep.day}
Night: {night_label}

Core overnight sleep:
  Total sleep: {seconds_to_hm(core_sleep.total_sleep_duration)}
  Sleep efficiency: {core_sleep.efficiency} %

Supplemental sleep (naps):
  Total: {seconds_to_hm(supplemental_seconds)}

Total sleep (24h): {seconds_to_hm(total_24h_seconds)}

Sleep score: {daily_sleep.score}
Readiness score: {readiness.score}
""".strip()

print(summary)

🛏️ Sleep Summary — 2026-01-11
Night: Jan 10 → Jan 11

Core overnight sleep:
  Total sleep: 4h 42m
  Sleep efficiency: 87 %

Supplemental sleep (naps):
  Total: 5h 49m

Total sleep (24h): 10h 31m

Sleep score: 76
Readiness score: 79


In [10]:
from datetime import timedelta


def seconds_to_hm(seconds: int | None) -> str:
    if not seconds or seconds <= 0:
        return "0h 00m"
    h = seconds // 3600
    m = (seconds % 3600) // 60
    return f"{h}h {m:02d}m"


def seconds_to_minutes(seconds: int | None) -> str:
    if seconds is None:
        return "n/a"
    return f"{seconds // 60}"


# ------------------------------------------------------------------
# Core objects (already selected upstream)
# ------------------------------------------------------------------

sd = sleep_doc  # Core SleepDocument
ds = daily_sleep  # DailySleepScore
dr = readiness  # DailyReadinessScore

# These must already exist from your earlier logic
# supplemental_seconds
# total_24h_seconds


# ------------------------------------------------------------------
# Derived metrics (core sleep)
# ------------------------------------------------------------------

total_sleep = seconds_to_hm(sd.total_sleep_duration)
time_in_bed = seconds_to_hm(sd.time_in_bed)
latency_min = seconds_to_minutes(sd.latency)

rem_hm = seconds_to_hm(sd.rem_sleep_duration)
deep_hm = seconds_to_hm(sd.deep_sleep_duration)

rem_pct = (
    round(100 * sd.rem_sleep_duration / sd.total_sleep_duration)
    if sd.rem_sleep_duration and sd.total_sleep_duration
    else "n/a"
)
deep_pct = (
    round(100 * sd.deep_sleep_duration / sd.total_sleep_duration)
    if sd.deep_sleep_duration and sd.total_sleep_duration
    else "n/a"
)

# DailySleepScore uses flattened fields
timing_label = "Optimal" if ds.timing >= 90 else f"{ds.timing}/100"


# ------------------------------------------------------------------
# Final formatted summary
# ------------------------------------------------------------------

summary = f"""
🛏️ Sleep Summary — {sd.day}
Night: {sd.bedtime_start.strftime("%b %d")} → {sd.bedtime_end.strftime("%b %d")}

Core overnight sleep:
  Total sleep: {seconds_to_hm(sd.total_sleep_duration)}
  Sleep efficiency: {sd.efficiency} %

Supplemental sleep (naps):
  Total: {seconds_to_hm(supplemental_seconds)}

Total sleep (24h): {seconds_to_hm(total_24h_seconds)}

────────────────────────────

Time in bed: {time_in_bed}
Latency: {latency_min} min

REM sleep: {rem_hm} ({rem_pct}%)
Deep sleep: {deep_hm} ({deep_pct}%)

Average HR: {sd.average_heart_rate:.0f} bpm
Lowest HR: {sd.lowest_heart_rate} bpm
Average HRV: {sd.average_hrv} ms

Sleep score: {ds.score}
Timing: {timing_label}
Readiness score: {dr.score}
""".strip()

print(summary)

NameError: name 'sleep_doc' is not defined